In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),  # LRN层
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),  # LRN层
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [4]:
# 训练模型
def train(model, train_loader, criterion, optimizer, num_epochs, device):
    model.train()
    loss_list = []
    for epoch in range(num_epochs):
        loss_list.append([])
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0:
                print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}')
                loss_list[-1].append(loss.item())
    return loss_list


# 测试模型
def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%')

In [ ]:
# 设置随机数种子
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

# 设置设备
if torch.cuda.is_available():
    device = torch.device('cuda')
# elif torch.backends.mps.is_available():
#     device = torch.device('mps')
else:
    device = torch.device('cpu')

transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=10)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=10)

model = AlexNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)
loss_list = train(model, trainloader, criterion, optimizer, 25, device)

100%|██████████| 170M/170M [00:10<00:00, 15.6MB/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 0, Batch 0, Loss: 2.3025145530700684
Epoch 0, Batch 100, Loss: 1.8794265985488892
Epoch 0, Batch 200, Loss: 1.476253867149353
Epoch 0, Batch 300, Loss: 1.2999252080917358
Epoch 1, Batch 0, Loss: 1.448592185974121
Epoch 1, Batch 100, Loss: 1.1500862836837769
Epoch 1, Batch 200, Loss: 1.1847513914108276
Epoch 1, Batch 300, Loss: 1.2623569965362549
Epoch 2, Batch 0, Loss: 1.0048755407333374
Epoch 2, Batch 100, Loss: 0.7863324880599976
Epoch 2, Batch 200, Loss: 0.9242383241653442
Epoch 2, Batch 300, Loss: 0.8784202933311462
Epoch 3, Batch 0, Loss: 0.923408031463623
Epoch 3, Batch 100, Loss: 0.8275366425514221
Epoch 3, Batch 200, Loss: 0.8603533506393433
Epoch 3, Batch 300, Loss: 0.7879496216773987
Epoch 4, Batch 0, Loss: 0.7173369526863098
Epoch 4, Batch 100, Loss: 0.6450088024139404
Epoch 4, Batch 200, Loss: 0.7726433873176575
Epoch 4, Batch 300, Loss: 0.7827979326248169
Epoch 5, Batch 0, Loss: 

TypeError: test() missing 1 required positional argument: 'device'

In [6]:
test(model, testloader, criterion, device)

Test Loss: 0.0058, Accuracy: 80.43%


In [9]:
torch.save(model.state_dict(), 'AlexNet.pth')